# Modelig script
## import packages and load data

In [ ]:
! pip install boruta

In [ ]:
!pip install xlsxwriter

In [ ]:
! pip install rasterio 

In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import math
import glob
from datetime import datetime
from scipy import stats
import rasterio as rio
#from boruta import BorutaPy

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
path = '/content/drive/MyDrive/Thesis/Data/'
data = pd.read_csv(os.path.join(path, 'cleaned_pixel_Traindata.csv'))
test = pd.read_csv(os.path.join(path, 'AlaskaWater_16_200722.csv'))

data.columns

Index(['Unnamed: 0', 'VV', 'VH', 'angle', 'ice', 'lake', 'windU', 'windV',
       'cont', 'corr', 'ent', 'VVwindowVar', 'VHwindowVar', 'pixelId', 'imgId',
       'Date', 'country', 'month', 'VV/VH', 'windRes', 'windDir', 'year',
       'VVnorm', 'VHnorm'],
      dtype='object')

In [54]:
alaska = data[data['country'] == 'Canada']
np.sort(alaska.loc[alaska['ice'] == 1, 'imgId'].unique())

array([ 0,  1,  3,  4,  5,  7,  8,  9, 10, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 28, 29, 33, 34, 35, 37, 38, 39])

## Create a validation set for feature selection

In [ ]:
train, val = train_test_split(data, test_size = 0.2, random_state = 42) 

print('shape of trainset', train.shape)
print('shpe of validationset', val.shape)

shape of trainset (527783, 24)
shpe of validationset (131946, 24)


## Boruta feature selection

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np

X = val[['VVnorm', 'VHnorm', 'VV/VH', 'angle' ,'cont', 'corr', 'ent', 'VVwindowVar', 'VHwindowVar', 'windDir', 'windRes']]
y = val['ice']

###initialize Boruta
forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5
)
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)
### fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(np.array(X), np.array(y))
### print results
green_area = X.columns[boruta.support_].to_list()
blue_area = X.columns[boruta.support_weak_].to_list()



In [ ]:
# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(boruta.support_)):
    if boruta.support_[i]:
        print("Passes the test.     : ", X.columns[i],
              " - Ranking: ", boruta.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              X.columns[i], " - Ranking: ", boruta.ranking_[i])

## Feature slection based on feature importance of three

In [122]:
feature_names = ['VV normalized', 'VH normalized', 'VV/VH', 'Angle' ,'Contrast', 'Correlation', 
                 'Entropy', 'VV window variance', 'VH window variance', 'Wind direction', 'Wind speed']
colnames = ['VVnorm', 'VHnorm', 'VV/VH', 'angle' ,'cont', 'corr', 'ent', 'VVwindowVar', 'VHwindowVar', 'windDir', 'windRes']
labels = val['ice']
features = val[colnames]

train_X, test_X, train_y, test_y = train_test_split(features, labels, test_size = 0.2)


rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 10)
rf.fit(train_X, train_y)

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
forest_importances = pd.Series(importances, index=feature_names)
forest_importance = pd.DataFrame({'features':feature_names, 
                                  'importances': importances,
                                  'columns': colnames}).sort_values('importances', ascending=False)


In [ ]:
forest_importance.loc[forest_importance['importances'] > 0.05, 'pallete'] = 'darkgreen'
forest_importance.loc[forest_importance['importances'] <= 0.05, 'pallete'] = 'firebrick'
my_pal = list(forest_importance['pallete'])

fig, ax = plt.subplots(1,1, figsize=(6,4))

sns.barplot(data = forest_importance, x='importances', y = 'features', palette = my_pal, ax = ax)

ax.set(xlabel='mean decrease in impurity (gini)')
plt.show()

fig.savefig('RFimportance.png')

In [124]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    rf, test_X, test_y, n_repeats=50, random_state=42, n_jobs=2
)

result = result.importances_mean

In [ ]:
forest_importance_permutation = pd.DataFrame({'features':feature_names, 
                                              'importances': result,
                                              'columns': colnames}).sort_values('importances', ascending=False)

forest_importance_permutation.loc[forest_importance_permutation['importances'] > 0.004, 'palette'] = 'darkgreen'
forest_importance_permutation.loc[forest_importance_permutation['importances'] <= 0.004, 'palette'] = 'firebrick'

my_pal = list(forest_importance_permutation['palette'])

fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.barplot(data = forest_importance_permutation, x='importances', y = 'features', palette = my_pal, ax = ax)

ax.set(xlabel='mean accuracy loss')
plt.show()

fig.savefig('permutationimportance.png')

## Random Forest for all feature combinations  
### Train and test on all study areas simultaniously 

In [ ]:
# Set feature combinations
importance = list(forest_importance.loc[forest_importance['importances'] > 0.08, 'columns'])
importance_permutation = list(forest_importance_permutation.loc[forest_importance_permutation['importances'] > 0.004, 'columns'])
sar = ['VVnorm', 'VHnorm', 'VV/VH', 'angle']
spatial = ['VVnorm', 'VHnorm', 'VV/VH', 'angle', 'cont', 'corr', 'ent', 'VVwindowVar', 'VHwindowVar']
wind = ['VVnorm', 'VHnorm', 'VV/VH', 'angle', 'windRes', 'windDir']
all = ['VVnorm', 'VHnorm', 'VV/VH', 'angle', 'cont', 'corr', 'ent', 'VVwindowVar', 'VHwindowVar', 'windRes', 'windDir']
print(importance)
FCs = [importance,importance_permutation,sar,spatial,wind,all]
names = ['RF Importance','Importance permutation', 'Sentinel-1 SAR' ,'Texture/spatial','Wind', 'all']

study_areas = ['Alaska', 'Canada', 'Finland', 'Russia']


['VVnorm', 'VV/VH', 'VHnorm', 'cont']


In [ ]:
FC = []
testing =[]
accuracy = []
recall = []
precision = []
f1_score = []

for cols, name in zip(FCs, names):
  print('Training on:', name)
  X = train[cols]
  y = train['ice']
  train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) 
  
  rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 5)
  rf.fit(train_X, train_y)

  pred = rf.predict(test_X)

  tn, fp, fn, tp = confusion_matrix(test_y, pred).ravel()
  
  acc = (tn+tp)/(tn+tp+fp+fn)
  sens = tp/(tp+fn)
  spec = tn/(tn+fp)
  f1 = 2*((spec * sens)/(spec + sens))

  FC.append(name)
  testing.append('all')
  accuracy.append(acc)
  recall.append(sens)
  precision.append(spec)
  f1_score.append(f1)

FC_results_all = pd.DataFrame({'Features': FC,
                               'Testing': testing,
                               'accuracy':accuracy,
                               'recall':recall,
                               'precision':precision,
                               'f1_score':f1_score})



Training on: RF Importance
Training on: Importance permutation
Training on: Sentinel-1 SAR
Training on: Texture/spatial
Training on: Wind
Training on: all


In [ ]:
print(FC_results_all)

                 Features Testing  accuracy    recall  precision  f1_score
0           RF Importance     all  0.698229  0.750256   0.646165  0.694331
1  Importance permutation     all  0.700370  0.745142   0.655566  0.697490
2          Sentinel-1 SAR     all  0.699546  0.753608   0.645444  0.695345
3         Texture/spatial     all  0.698447  0.747869   0.648989  0.694929
4                    Wind     all  0.705259  0.783420   0.627040  0.696561
5                     all     all  0.700247  0.763912   0.636536  0.694431


### Leave one study area out validation

In [ ]:
FC = []
testing =[]
accuracy = []
recall = []
precision = []
f1_score = []

for cols, name in zip(FCs, names):
  print('Training on', name)
  
  for sa in study_areas:
    print('Testing for on', sa)
    train_country = train[train['country'] != sa]
    test_country = train[train['country'] == sa]

    train_country_X = train_country[cols]
    train_country_y = train_country['ice']

    test_country_X = test_country[cols]
    test_country_y = test_country['ice']

    rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 5)
    rf.fit(train_country_X, train_country_y)
    pred = rf.predict(test_country_X)

    tn, fp, fn, tp = confusion_matrix(test_country_y, pred).ravel()
  
    acc = (tn+tp)/(tn+tp+fp+fn)
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    f1 = 2*((spec * sens)/(spec + sens))

    FC.append(name)
    testing.append(sa)
    accuracy.append(acc)
    recall.append(sens)
    precision.append(spec)
    f1_score.append(f1)

FC_results_sa = pd.DataFrame({'Features': FC,
                              'Testing': testing,
                              'accuracy':accuracy,
                              'recall':recall,
                              'precision':precision,
                              'f1_score':f1_score})

Training on RF Importance
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia
Training on Importance permutation
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia
Training on Sentinel-1 SAR
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia
Training on Texture/spatial
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia
Training on Wind
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia
Training on all
Testing for on Alaska
Testing for on Canada
Testing for on Finland
Testing for on Russia


In [ ]:
print(FC_results_sa)

                  Features  Testing  accuracy    recall  precision  f1_score
0            RF Importance   Alaska  0.581807  0.752731   0.380276  0.505285
1            RF Importance   Canada  0.673121  0.538504   0.776065  0.635819
2            RF Importance  Finland  0.728707  0.822268   0.626135  0.710922
3            RF Importance   Russia  0.734753  0.727188   0.741925  0.734483
4   Importance permutation   Alaska  0.585079  0.801425   0.329994  0.467494
5   Importance permutation   Canada  0.685648  0.543073   0.794678  0.645214
6   Importance permutation  Finland  0.742396  0.808017   0.670453  0.732836
7   Importance permutation   Russia  0.735079  0.737424   0.732855  0.735133
8           Sentinel-1 SAR   Alaska  0.585410  0.802111   0.329907  0.467522
9           Sentinel-1 SAR   Canada  0.673676  0.532612   0.781551  0.633503
10          Sentinel-1 SAR  Finland  0.726669  0.814381   0.630509  0.710745
11          Sentinel-1 SAR   Russia  0.738008  0.736365   0.739565  0.737962

### Train and test locally

In [ ]:
FC = []
testing =[]
accuracy = []
recall = []
precision = []
f1_score = []

for sa in study_areas:
  print('train and test on:', sa)
  SA_data = train[train['country'] == sa]

  for cols, name in zip(FCs, names):
    print('Training on', name)
    X = SA_data[cols]
    y = SA_data['ice']

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) 

    rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 5)
    rf.fit(train_X, train_y)

    pred = rf.predict(test_X)

    tn, fp, fn, tp = confusion_matrix(test_y, pred).ravel()
  
    acc = (tn+tp)/(tn+tp+fp+fn)
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    f1 = 2*((spec * sens)/(spec + sens))

    FC.append(name)
    testing.append(sa)
    accuracy.append(acc)
    recall.append(sens)
    precision.append(spec)
    f1_score.append(f1)

FC_results_local = pd.DataFrame({'Features': FC,
                                 'Testing': testing,
                                 'accuracy':accuracy,
                                 'recall':recall,
                                 'precision':precision,
                                 'f1_score':f1_score})




train and test on: Alaska
Training on RF Importance
Training on Importance permutation
Training on Sentinel-1 SAR
Training on Texture/spatial
Training on Wind
Training on all
train and test on: Canada
Training on RF Importance
Training on Importance permutation
Training on Sentinel-1 SAR
Training on Texture/spatial
Training on Wind
Training on all
train and test on: Finland
Training on RF Importance
Training on Importance permutation
Training on Sentinel-1 SAR
Training on Texture/spatial
Training on Wind
Training on all
train and test on: Russia
Training on RF Importance
Training on Importance permutation
Training on Sentinel-1 SAR
Training on Texture/spatial
Training on Wind
Training on all


In [ ]:
FC_results_local

,Features,Testing,accuracy,recall,precision,f1_score
0,RF Importance,Alaska,0.648015,0.740093,0.540155,0.624512
1,Importance permutation,Alaska,0.625182,0.662326,0.581672,0.619384
2,Sentinel-1 SAR,Alaska,0.648517,0.738791,0.542770,0.625789
3,Texture/spatial,Alaska,0.645757,0.733860,0.542552,0.623870
4,Wind,Alaska,0.771817,0.815349,0.720824,0.765178
5,all,Alaska,0.754705,0.806884,0.693582,0.745955
6,RF Importance,Canada,0.711490,0.619479,0.781934,0.691291
7,Importance permutation,Canada,0.693126,0.681033,0.702385,0.691544
8,Sentinel-1 SAR,Canada,0.694051,0.680606,0.704345,0.692272
9,Texture/spatial,Canada,0.706171,0.603264,0.784956,0.682220


In [ ]:
import xlsxwriter
with pd.ExcelWriter('RFresultsMaxLeafs5.xlsx', engine='xlsxwriter') as writer:
    FC_results_all.to_excel(writer, sheet_name='All SAs')
    FC_results_sa.to_excel(writer, sheet_name='Leave 1 SA out')
    FC_results_local.to_excel(writer, sheet_name='Local')

## Image testing

In [110]:
test = pd.read_csv(os.path.join(path, 'CanadaWater_36_190822.csv'))
test['VVnorm'] = stats.zscore(test['VV'])
test['VHnorm'] = stats.zscore(test['VH'])

importantFeatures = ['VVnorm', 'windRes', 'windDir', 'cont', 'angle', 'corr']

In [111]:
#importantFeatures = ['VVnorm', 'windRes', 'windDir', 'cont', 'angle', 'corr']
X_test = test[importantFeatures]

X_train = train[importantFeatures]
y_train = train['ice']

rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 10)
rf.fit(X_train, y_train)

pred_sim = rf.predict(X_test)
test['pred_sim'] = pred_sim

In [112]:
X_train = train.loc[train['country'] == 'Canada', importantFeatures]
y_train = train.loc[train['country'] == 'Canada', 'ice']

X_test = test[importantFeatures]

rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 10)
rf.fit(X_train, y_train)

pred_local = rf.predict(X_test)
test['pred_local'] = pred_local

In [113]:
X_train = train.loc[train['country'] != 'Canada', importantFeatures]
y_train = train.loc[train['country'] != 'Canada', 'ice']

X_test = test[importantFeatures]

rf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 10)
rf.fit(X_train, y_train)

pred_loso = rf.predict(X_test)
test['pred_loso'] = pred_loso

In [114]:
test_img_pred = test[['pred_sim', 'pred_local', 'pred_loso','pixelId']]

In [115]:
n = 2000*2000
id = pd.DataFrame({'pixelId':np.arange(n)})
id_pred_merge = pd.merge(id, test_img_pred, on ='pixelId', how = 'left')

In [116]:
pred_sim_array = np.array(id_pred_merge['pred_sim'])
pred_local_array = np.array(id_pred_merge['pred_local'])
pred_loso_array = np.array(id_pred_merge['pred_loso'])
pred_local_array[pred_local_array==0].shape

(238700,)

In [117]:
pred_sim_array = pred_sim_array.reshape(1,2000,2000)
pred_local_array = pred_local_array.reshape(1,2000,2000)
pred_loso_array = pred_loso_array.reshape(1,2000,2000)

In [118]:
prediction_bands = np.concatenate([pred_sim_array,pred_local_array, pred_loso_array], axis = 0)

In [119]:
orgIm = rio.open('/content/SarExportLakeImgs_CanadaWater_36_190822.tif')

In [120]:
meta = orgIm.meta
meta.update({'count':3})
meta

{'count': 3,
 'crs': CRS.from_epsg(3857),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 2000,
 'nodata': None,
 'transform': Affine(8.627349975766615, 0.0, -12298267.635324758,
       0.0, -8.627349975766148, 7275857.40583455),
 'width': 2000}

In [121]:
# Write the mosaic raster to disk
with rio.open('/content/CanadaWater_36_190822_predictions.tif', "w", **meta) as dest:
    dest.write(prediction_bands)